# Tubi Hackathon

## Liga v.s. Rikai
+ Rikai: ML_PREDICT/MLflow/ModelType/Pytorch/Vision UDF and UDT/Rikai format
+ Liga: ML_PREDICT/MLflow/ModelType/Sklearn

## What we have done!
+ We created 33 merged pull requests to release liga v0.2.0.dev3.
+ It is now lightweighted compared to Rikai

# The Dataset
# The breast cancer dataset

In [1]:
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y=True)
(X.shape, y.shape)

((569, 30), (569,))

In [2]:
[(X[0], y[0]), (X[568], y[568])]

[(array([1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
         3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
         8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
         3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
         1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01]),
  0),
 (array([7.760e+00, 2.454e+01, 4.792e+01, 1.810e+02, 5.263e-02, 4.362e-02,
         0.000e+00, 0.000e+00, 1.587e-01, 5.884e-02, 3.857e-01, 1.428e+00,
         2.548e+00, 1.915e+01, 7.189e-03, 4.660e-03, 0.000e+00, 0.000e+00,
         2.676e-02, 2.783e-03, 9.456e+00, 3.037e+01, 5.916e+01, 2.686e+02,
         8.996e-02, 6.444e-02, 0.000e+00, 0.000e+00, 2.871e-01, 7.039e-02]),
  1)]

# ML Engineers
## Train the model and register it on MLflow

In [3]:
import getpass

import mlflow
from liga.sklearn.mlflow import log_model
from sklearn.ensemble import RandomForestClassifier


mlflow_tracking_uri = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(mlflow_tracking_uri)

# train a model
with mlflow.start_run() as run:
    ####
    # Part 1: Train the model and register it on MLflow
    ####
    model = RandomForestClassifier(max_depth=2, random_state=0)
    model.fit(X, y)

    registered_model_name = f"{getpass.getuser()}_random_forest_clf"
    log_model(model, registered_model_name=registered_model_name)

/home/da/.cache/pants/named_caches/pex_root/venvs/5e9cb7b4f75e723700e4da15663db04ffd281f57/3ee47a02dc94d67e7a24c8eac671fbacfe134bce/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2023/01/20 14:59:12 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'da_random_forest_clf' already exists. Creating a new version of this model...
2023/01/20 14:59:12 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: da_random_forest_clf, version 2
Created version '2' of model 'da_random_forest_clf'.
/home/da/.cache/pants/named_caches/pex_root/venvs/5e9cb7b4f75e723700e4da15663db04ffd281f57/3ee47a02dc94d67e7a24c8eac671fbacfe134bce/lib/python3.9/site-packages/liga/mlflow/logger.py:137: 

# SQL Users
# ML_PREDICT via ML-enhanced Spark SQL

In [4]:
from liga.spark import init
spark = init()

2023-01-20 14:59:24,761 WARNING Rikai (__init__.py:52): Jar type `github` is for stable release, it may fail when version contains dev
2023-01-20 14:59:24,762 INFO Rikai (__init__.py:107): Set `spark.jars` to https://github.com/liga-ai/liga/releases/download/v0.2.0.dev11/liga-spark321-assembly_2.12-0.2.0.dev11.jar
23/01/20 14:59:25 WARN Utils: Your hostname, tubi resolves to a loopback address: 127.0.1.1; using 192.168.31.32 instead (on interface wlp0s20f3)
23/01/20 14:59:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/20 14:59:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from liga.mlflow import CONF_MLFLOW_TRACKING_URI
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set(CONF_MLFLOW_TRACKING_URI, mlflow_tracking_uri)
spark.sql(f"""
CREATE OR REPLACE MODEL mlflow_sklearn_m USING 'mlflow:///{registered_model_name}';
"""
)

spark.sql("show models").show(1, vertical=False, truncate=False)

+----------------+------+------------------------------+-------+
|name            |flavor|uri                           |options|
+----------------+------+------------------------------+-------+
|mlflow_sklearn_m|      |mlflow:///da_random_forest_clf|       |
+----------------+------+------------------------------+-------+



In [6]:
result = spark.sql(f"""
select ML_PREDICT(mlflow_sklearn_m, array(1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
        3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
        8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
        3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
        1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01))
"""
)

result.printSchema()
result.toPandas()

root
 |-- mlflow_sklearn_m: integer (nullable = true)



,mlflow_sklearn_m
0,0


In [7]:
result = spark.sql(f"""
select ML_PREDICT(mlflow_sklearn_m, array(7.760e+00, 2.454e+01, 4.792e+01, 1.810e+02, 5.263e-02, 4.362e-02,
        0.000e+00, 0.000e+00, 1.587e-01, 5.884e-02, 3.857e-01, 1.428e+00,
        2.548e+00, 1.915e+01, 7.189e-03, 4.660e-03, 0.000e+00, 0.000e+00,
        2.676e-02, 2.783e-03, 9.456e+00, 3.037e+01, 5.916e+01, 2.686e+02,
        8.996e-02, 6.444e-02, 0.000e+00, 0.000e+00, 2.871e-01, 7.039e-02))
"""
)

result.printSchema()
result.toPandas()

root
 |-- mlflow_sklearn_m: integer (nullable = true)



,mlflow_sklearn_m
0,1
